In [80]:
import pandas as pd
import numpy as np

from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor

In [81]:
use_descriptors = True
use_fingerprints = False

regression = False

### Retrieving data the old way:

In [82]:
### Input standard SMILES column
def CalculateMorganFingerprint(mol):
    mol = mol.apply(Chem.MolFromSmiles)
    mfpgen = AllChem.GetMorganGenerator(radius=2,fpSize=2048)
    fingerprint = np.array([mfpgen.GetFingerprintAsNumPy(x) for x in mol])
    fingerprint = pd.DataFrame(fingerprint, columns = ['mfp'+str(i) for i in range(fingerprint.shape[1])])
    return fingerprint

### Input standard SMILES column
def CalculateDescriptors(mol):
    mol = mol.apply(Chem.MolFromSmiles)
    calc = Calculator(descriptors, ignore_3D=False)
    X_mordred = calc.pandas(mol, nproc=1)
    X_mordred = X_mordred.select_dtypes(['number'])
    #normalize
    X_mordred = (X_mordred-X_mordred.min())/(X_mordred.max()-X_mordred.min())
    #drop columns wth low std
    X_mordred = X_mordred.loc[:,X_mordred.std()>0.01]
    return X_mordred

In [83]:
def Load_downloaded_CSV_BACE(path, regression = False):
    df = pd.read_csv(path)
    df.drop_duplicates('mol')
    df = df.dropna()
    #df.drop(['CID', 'canvasUID'], axis=1, inplace=True)

    if regression:
        df['Target'] = df['pIC50']
        df.drop('Class', axis=1, inplace=True)
        df.drop('pIC50', axis=1, inplace=True)
    else:
        df['Target'] = df['Class']
        df.drop('Class', axis=1, inplace=True)
        df.drop('pIC50', axis=1, inplace=True)

    df = df[['mol', 'Target', 'Model']]

    if use_descriptors:
        new_df = CalculateDescriptors(df['mol'])
    if use_fingerprints:
        new_df = CalculateMorganFingerprint(df['mol'])
        
    new_df['Target'] = df['Target']
    new_df['Model'] = df['Model']

    return new_df

def Split_downloaded_CSV_BACE(df, scaffold=True):
    if not scaffold:
        X = df.drop(['Target', 'Model'], axis=1)
        y = df[['Target']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.111, random_state=42)
        return X_train, y_train, X_test, y_test, X_valid, y_valid

    X = df.drop(['Target'], axis=1)
    y = df[['Target', 'Model']]

    X_train = X[X['Model'] == 'Train']
    y_train = y[y['Model'] == 'Train']
    X_test = X[X['Model'] == 'Test']
    y_test = y[y['Model'] == 'Test']
    X_valid = X[X['Model'] == 'Valid']
    y_valid = y[y['Model'] == 'Valid']
    
    X_train = X_train.drop('Model', axis=1)
    y_train = y_train.drop('Model', axis=1)
    X_test = X_test.drop('Model', axis=1)
    y_test = y_test.drop('Model', axis=1)
    X_valid = X_valid.drop('Model', axis=1)
    y_valid = y_valid.drop('Model', axis=1)
    
    return X_train, y_train, X_test, y_test, X_valid, y_valid

In [84]:
df_classification = Load_downloaded_CSV_BACE(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\BACE\bace.csv")

c:\Users\wojci\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 1513/1513 [04:24<00:00,  5.73it/s]
C:\Users\wojci\AppData\Local\Temp\ipykernel_9508\898515923.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['Target'] = df['Target']
C:\Users\wojci\AppData\Local\Temp\ipykernel_9508\898515923.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['Model'] =

In [85]:
scaffold_split = True

X_train, y_train, X_test, y_test, X_valid, y_valid = Split_downloaded_CSV_BACE(df_classification, scaffold=scaffold_split)

In [86]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(203, 1222)
(1265, 1222)
(45, 1222)


### Configurations

In [87]:
def model_builder(model_name, hyperparams):
    if model_name == 'rf':
        if regression:
            model = RandomForestRegressor(n_estimators=hyperparams["n_estimators"],
                                    min_samples_split=hyperparams["min_samples_split"],
                                    criterion=hyperparams["criterion"],
                                    bootstrap=hyperparams["bootstrap"])
        else:
            model = RandomForestClassifier(n_estimators=hyperparams["n_estimators"],
                                    min_samples_split=hyperparams["min_samples_split"], 
                                    criterion=hyperparams["criterion"],
                                    bootstrap=hyperparams["bootstrap"])

    return model
    

In [144]:
def train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid, metrics=[], iterations=1, regression=False):
    for i in range(iterations):
        model.fit(X_train, y_train)
        
        y_test_predicted = model.predict(X_test)
        y_valid_predicted = model.predict(X_valid)

        roc_auc_test = roc_auc_score(y_test, y_test_predicted)
        roc_auc_valid = roc_auc_score(y_valid, y_valid_predicted)

        print(roc_auc_test)
        print(roc_auc_valid)

def benchmark_train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid, metrics=[], iterations=1, regression=False):
    model = dc.models.SklearnModel(model) ### for benchmark 
    for i in range(iterations):
        #print(X_train.to_numpy().shape)
        #print(y_train.to_numpy().shape)
        #print(np.ones_like(y_train).shape)
        #print(np.array([i for i in range(y_train.shape[0])]).shape)

        train_set = dc.data.DiskDataset.from_numpy(X=X_train.to_numpy(), y=y_train.to_numpy(), w=np.ones_like(y_train), ids=np.array([i for i in range(y_train.shape[0])]), tasks=['Class'])
        test_set = dc.data.DiskDataset.from_numpy(X=X_test.to_numpy(), y=y_test.to_numpy(), w=np.ones_like(y_test), ids=np.array([i for i in range(y_test.shape[0])]), tasks=['Class'])
        valid_set = dc.data.DiskDataset.from_numpy(X=X_valid.to_numpy(), y=y_valid.to_numpy(), w=np.ones_like(y_valid), ids=np.array([i for i in range(y_valid.shape[0])]), tasks=['Class'])

        model.fit(train_set)
        
        metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
        test_metric = model.evaluate(test_set, [metric]) #, transformers)
        valid_metric = model.evaluate(valid_set, [metric]) #, transformers)

        print(test_metric)
        print(valid_metric)

In [145]:
models = ['rf']

hyperparams = {
  "bootstrap": True,
  "criterion": "entropy",
  "min_samples_split": 32,
  "n_estimators": 30
}

In [146]:
for model_name in models:
    model = model_builder(model_name, hyperparams)
    
    benchmark_train_and_test(model, X_train, y_train, X_test, y_test, X_valid, y_valid)
    

{'mean-roc_auc_score': 0.7940821348331779}
{'mean-roc_auc_score': 0.5909090909090908}
